In [1]:
import numpy as np
import os
import csv
from pickle import load, dump

In [2]:
with open('game_data_public.STX.PremierDraft.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)

In [3]:
x = 0
card_dict = {}
card_index = {}
drafted_cards = []
for index, word in enumerate(data[0]):
    if word.startswith('deck_'):
        card = word.split("deck_")[1]
        print(card)
        card_dict[card] = x
        card_index[x] = card
        x = x + 1
        drafted_cards.append(index)
    if word.startswith("sideboard_"):
        drafted_cards.append(index)
print(x)
dump(card_dict, open('cardtoindex.pkl', 'wb'))
dump(card_index, open('indextocard.pkl', 'wb'))

In [4]:
draftID = "0x"
clean_data = []
wins = 0
losses = 0
for index, value in enumerate(data[1:], 1):
    if value[2] != draftID:
        draftID = value[2]
        wins = 0
        losses = 0
        draft_data = {}
    if value[2] == draftID:
        if value[15] == "True":
            wins = wins + 1
        else:
            losses = losses + 1
        if index == len(data) - 1 or data[index + 1][2] != draftID:
            draft_data = {"DraftID": draftID}
            for c_index in drafted_cards:
                if int(value[c_index]) > 0:
                    card_name = card_index[(c_index - 702) % 342]
                    if card_name == "Mountain" or card_name == "Island" or card_name == "Plains" or card_name == "Swamp" or card_name == "Forest":
                        continue
                    if c_index < 1045:
                        draft_data["main_" + card_name] = value[c_index]
                    if "draft_" + card_name in draft_data:
                        draft_data["draft_" + card_name] = int(draft_data["draft_" + card_name]) + int(value[c_index])
                    else:
                        draft_data["draft_" + card_name] = value[c_index]
            draft_data["Wins"] = wins
            draft_data["Losses"] = losses
            clean_data.append(draft_data)

In [5]:
winning_data = []
for values in clean_data:
    if values["Wins"] > 2:
        winning_data.append(values)
dump(winning_data, open('winning_data.pkl', 'wb'))

In [ ]:
if not os.path.exists("data"):
    os.mkdir("data")


In [ ]:
def datasetup(draft):
    cards = 0
    alldrafted = np.zeros(343, np.int8)
    maindeck = np.zeros(343, np.int8)
    for key, value in draft.items():
        if key.startswith("main_"):
            cards = cards + int(value)
            card = key.split("main_")[1]
            maindeck[int(card_dict[card])] = value
        if key.startswith("draft_"):
            card = key.split("draft_")[1]
            alldrafted[int(card_dict[card])] = value
    return cards, alldrafted, maindeck

In [ ]:
for index, deck in enumerate(winning_data):
    cardcount, alldrafted, removedmain = datasetup(deck)
    y = removedmain
    X1 = alldrafted
    np.save('data/' + str(index) + '.npy', np.array(([X1], [y])))